In [78]:
from __future__ import print_function

import numpy as np
import pandas as pd

from datetime import datetime

import networkx as nx


import statsmodels.api as sm


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns

from sklearn.metrics import confusion_matrix


from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE

from sklearn.metrics import make_scorer

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression

from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import classification_report


from sklearn import datasets
from sklearn import linear_model

from sklearn.model_selection import GridSearchCV

from sklearn import metrics


In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df_el = pd.read_pickle('./df.pkl')

fraud = pd.read_pickle('./fraud.pkl')

nfraud = pd.read_pickle('./fraud.pkl')

In [5]:
data = pd.read_csv('../../../raw_data/provider_final_updated.csv')

In [6]:
df = data.copy()

In [7]:
df.columns

Index(['Unnamed: 0', 'Provider', 'PotentialFraud', 'DurationMedianIP',
       'DurationMeanIP', 'DurationMedianOP', 'DurationMeanOP', 'Serves',
       'ChronCondsMedian', 'ChronCondsMean', 'ChronConsMode', 'PtAgeMedian',
       'PtAgeMean', 'NumPtsIP', 'NumPtsOP', 'NumStatesIP', 'NumStatesOP',
       'SharePhysician', 'MergeAvgClaimReimbursed', 'NumTopCodeClaim',
       'DupClaimNum_IP', 'DupClaimRatio_IP', 'DupClaim_IP',
       'DupClaimStatesNum_IP', 'DupClaimMultiState_IP',
       'DupClaimProviderNum_IP', 'DupClaimMultiProvider_IP',
       'DupClaimStartDtNum_IP', 'DupClaimMultiStartDt_IP', 'DupClaimNum_OP',
       'DupClaimRatio_OP', 'DupClaim_OP', 'DupClaimStatesNum_OP',
       'DupClaimMultiState_OP', 'DupClaimProviderNum_OP',
       'DupClaimMultiProvider_OP', 'DupClaimStartDtNum_OP',
       'DupClaimMultiStartDt_OP', 'AvgDuration', 'NumOfPt', 'NumOfClm',
       'ClmPerPt', 'TotalRev', 'RevPerPt', 'ClmNoPhy', 'ClmAllPhy',
       'ClmNoProc', 'ClmPerDt'],
      dtype='object')

In [8]:
#df['ClmPerDt'].value_counts(normalize = True) * 100

In [9]:
df.head()

,Unnamed: 0,Provider,PotentialFraud,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,Serves,ChronCondsMedian,ChronCondsMean,...,AvgDuration,NumOfPt,NumOfClm,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt
0,0,PRV51001,No,4.0,5.000000,0.0,0.550000,Both,6.0,5.760000,...,1.44,24,25,1.04,109980,4582.50,0,2,23,1.09
1,1,PRV51003,Yes,4.0,5.161290,0.0,2.357143,Both,7.0,6.674242,...,3.67,117,132,1.13,671956,5743.21,0,4,93,1.19
2,2,PRV51004,No,0.0,0.000000,0.0,1.429530,OP,7.0,6.812081,...,1.43,138,149,1.08,52480,380.29,0,12,149,1.27
3,3,PRV51005,Yes,0.0,0.000000,0.0,1.088412,OP,7.0,6.886695,...,1.09,495,1165,2.35,284610,574.97,2,98,1165,3.27
4,4,PRV51007,No,5.0,5.333333,0.0,0.768116,Both,7.0,6.986111,...,0.96,58,72,1.24,36974,637.48,0,8,71,1.06


In [10]:
df = df.replace({'No': 0, 'Yes': 1})

In [11]:
data['Serves'].unique()

array(['Both', 'OP', 'IP'], dtype=object)

In [12]:
df = pd.get_dummies(df, columns = ['Serves'])

In [13]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [14]:
df.head()

,Provider,PotentialFraud,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,...,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt,Serves_Both,Serves_IP,Serves_OP
0,PRV51001,0,4.0,5.000000,0.0,0.550000,6.0,5.760000,5.0,79.0,...,1.04,109980,4582.50,0,2,23,1.09,1,0,0
1,PRV51003,1,4.0,5.161290,0.0,2.357143,7.0,6.674242,6.0,71.0,...,1.13,671956,5743.21,0,4,93,1.19,1,0,0
2,PRV51004,0,0.0,0.000000,0.0,1.429530,7.0,6.812081,8.0,72.0,...,1.08,52480,380.29,0,12,149,1.27,0,0,1
3,PRV51005,1,0.0,0.000000,0.0,1.088412,7.0,6.886695,5.0,70.0,...,2.35,284610,574.97,2,98,1165,3.27,0,0,1
4,PRV51007,0,5.0,5.333333,0.0,0.768116,7.0,6.986111,6.0,69.5,...,1.24,36974,637.48,0,8,71,1.06,1,0,0


In [15]:
data.isnull().sum().sort_values(ascending = False)

ClmPerDt                    0
ClmNoProc                   0
DupClaimRatio_IP            0
DupClaimNum_IP              0
NumTopCodeClaim             0
MergeAvgClaimReimbursed     0
SharePhysician              0
NumStatesOP                 0
NumStatesIP                 0
NumPtsOP                    0
NumPtsIP                    0
PtAgeMean                   0
PtAgeMedian                 0
ChronConsMode               0
ChronCondsMean              0
ChronCondsMedian            0
Serves                      0
DurationMeanOP              0
DurationMedianOP            0
DurationMeanIP              0
DurationMedianIP            0
PotentialFraud              0
Provider                    0
DupClaim_IP                 0
DupClaimStatesNum_IP        0
DupClaimMultiState_IP       0
DupClaimStartDtNum_OP       0
ClmAllPhy                   0
ClmNoPhy                    0
RevPerPt                    0
TotalRev                    0
ClmPerPt                    0
NumOfClm                    0
NumOfPt   

In [16]:
X = df.drop(['Provider', 'PotentialFraud'], axis = 1)

In [17]:
df_el = df_el.replace({'No': 0, 'Yes': 1})

In [18]:
X_el = df_el.drop(['Provider', 'PotentialFraud'], axis = 1)

In [19]:
X.head()

,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,PtAgeMean,NumPtsIP,...,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt,Serves_Both,Serves_IP,Serves_OP
0,4.0,5.000000,0.0,0.550000,6.0,5.760000,5.0,79.0,77.880000,5.0,...,1.04,109980,4582.50,0,2,23,1.09,1,0,0
1,4.0,5.161290,0.0,2.357143,7.0,6.674242,6.0,71.0,69.083333,53.0,...,1.13,671956,5743.21,0,4,93,1.19,1,0,0
2,0.0,0.000000,0.0,1.429530,7.0,6.812081,8.0,72.0,71.261745,0.0,...,1.08,52480,380.29,0,12,149,1.27,0,0,1
3,0.0,0.000000,0.0,1.088412,7.0,6.886695,5.0,70.0,69.526180,0.0,...,2.35,284610,574.97,2,98,1165,3.27,0,0,1
4,5.0,5.333333,0.0,0.768116,7.0,6.986111,6.0,69.5,68.305556,3.0,...,1.24,36974,637.48,0,8,71,1.06,1,0,0


In [20]:
X.shape

(5410, 47)

In [21]:
X.head()

,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,PtAgeMean,NumPtsIP,...,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt,Serves_Both,Serves_IP,Serves_OP
0,4.0,5.000000,0.0,0.550000,6.0,5.760000,5.0,79.0,77.880000,5.0,...,1.04,109980,4582.50,0,2,23,1.09,1,0,0
1,4.0,5.161290,0.0,2.357143,7.0,6.674242,6.0,71.0,69.083333,53.0,...,1.13,671956,5743.21,0,4,93,1.19,1,0,0
2,0.0,0.000000,0.0,1.429530,7.0,6.812081,8.0,72.0,71.261745,0.0,...,1.08,52480,380.29,0,12,149,1.27,0,0,1
3,0.0,0.000000,0.0,1.088412,7.0,6.886695,5.0,70.0,69.526180,0.0,...,2.35,284610,574.97,2,98,1165,3.27,0,0,1
4,5.0,5.333333,0.0,0.768116,7.0,6.986111,6.0,69.5,68.305556,3.0,...,1.24,36974,637.48,0,8,71,1.06,1,0,0


In [22]:
X.dtypes

DurationMedianIP            float64
DurationMeanIP              float64
DurationMedianOP            float64
DurationMeanOP              float64
ChronCondsMedian            float64
ChronCondsMean              float64
ChronConsMode               float64
PtAgeMedian                 float64
PtAgeMean                   float64
NumPtsIP                    float64
NumPtsOP                    float64
NumStatesIP                 float64
NumStatesOP                 float64
SharePhysician              float64
MergeAvgClaimReimbursed     float64
NumTopCodeClaim             float64
DupClaimNum_IP              float64
DupClaimRatio_IP            float64
DupClaim_IP                  object
DupClaimStatesNum_IP        float64
DupClaimMultiState_IP        object
DupClaimProviderNum_IP      float64
DupClaimMultiProvider_IP     object
DupClaimStartDtNum_IP       float64
DupClaimMultiStartDt_IP      object
DupClaimNum_OP              float64
DupClaimRatio_OP            float64
DupClaim_OP                 

In [23]:
X['DupClaimMultiStartDt_OP'].unique()

array(['yes', 'no'], dtype=object)

In [24]:
X['DupClaimMultiState_OP'].unique()

array(['yes', 'no'], dtype=object)

In [25]:
X['DupClaimNum_IP'].unique()

array([ 0.,  1.,  3.,  2.,  7., 10.,  9.,  4.,  6., 11.,  5.,  8., 22.,
       13., 15., 19.])

In [26]:
X['DupClaimNum_IP'].unique()

array([ 0.,  1.,  3.,  2.,  7., 10.,  9.,  4.,  6., 11.,  5.,  8., 22.,
       13., 15., 19.])

In [27]:
X['DupClaimMultiProvider_IP'].unique()

array(['no', 'yes'], dtype=object)

In [28]:
#X['DupClaimNum_OP'].unique()

In [29]:
X['DupClaim_OP'].unique()

array(['yes', 'no'], dtype=object)

In [30]:
X['DupClaimMultiState_OP'].unique()

array(['yes', 'no'], dtype=object)

In [31]:
X['DupClaimMultiProvider_OP'].unique()

array(['yes', 'no'], dtype=object)

In [32]:
X['DupClaimMultiStartDt_OP'].unique()

array(['yes', 'no'], dtype=object)

In [33]:
X = pd.get_dummies(X)

In [34]:
X.shape

(5410, 55)

In [35]:
X_el.head()

,AvgDuration,NumOfPt,NumOfClm,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt
0,3.67,117,132,1.13,671956,5743.21,0,4,93,1.19
1,1.09,495,1165,2.35,284610,574.97,2,98,1165,3.27
2,2.00,208,257,1.24,380322,1828.47,1,17,240,1.46
3,1.53,206,454,2.20,367754,1785.21,3,37,445,1.66
4,2.28,37,46,1.24,33146,895.84,0,3,45,1.10


In [36]:
y = df['PotentialFraud']

In [37]:
y_el = df_el['PotentialFraud']

In [38]:
X.to_pickle("./X.pkl")

In [39]:
y.to_pickle("./y.pkl")

## Pipelining

In [103]:
from sklearn.linear_model import LogisticRegression

In [105]:
#SMOTE?


In [106]:
resampling = SMOTE(random_state = 42)

In [107]:
model = LogisticRegression(solver = 'liblinear', \
                           penalty = 'l1',\
                           class_weight = 'balanced')

In [108]:
pipeline = Pipeline([('SMOTE', resampling), ('Logistic Regression', model)])

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,\
                        random_state = 0)

In [50]:
pipeline.fit(X_train, y_train)
pip_predict = pipeline.predict(X_test)

In [53]:
print("Classification report:\n", classification_report(y_test, pip_predict))

Classification report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.96      1474
           1       0.56      0.76      0.65       149

    accuracy                           0.92      1623
   macro avg       0.77      0.85      0.80      1623
weighted avg       0.94      0.92      0.93      1623



In [54]:
cm = metrics.confusion_matrix(y_test, pip_predict)

In [55]:
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[1387   87]
 [  36  113]]


In [72]:
probs = model.predict_proba(X_test)

In [75]:
roc_auc = metrics.roc_auc_score(y_test, probs[:,1])

In [80]:
print("ROC score is ", round(roc_auc, 3))

ROC score is  0.939


##  Linear Regression

In [119]:
from sklearn.linear_model import LogisticRegression

from sklearn.utils.class_weight import compute_class_weight

In [124]:
weighting = compute_class_weight('balanced', [0,1], y)

In [125]:
print('The weight for no fraud is', weighting[0].round(2))
print('The weight for potential fraud is', weighting[1].round(2))

The weight for no fraud is 0.55
The weight for potential fraud is 5.35


In [126]:
weights = {0:0.55, 1:5.35}

In [127]:
lgt = LogisticRegression(\
                         #l1_ratio = 1,
                        penalty = 'l1',
                         max_iter = 1000,
                         solver = 'liblinear', \
                         class_weight = weights)
                         #class_weight = "balanced")

In [128]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,\
                        random_state = 0, stratify = y)

In [129]:
grid = {"C": np.logspace(-4, 4, 20), "penalty": ["l1","l2"]}

In [130]:
scoring = {\
           'AUC': 'roc_auc',\
           'f1': 'f1'}

In [131]:
lgt_cv = GridSearchCV(lgt, grid, scoring = scoring,\
                      refit = 'AUC', \
                      return_train_score = True, \
                      cv = 10, n_jobs = -1)

In [132]:
lgt_cv.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=LogisticRegression(class_weight={0: 0.55, 1: 5.35},
                                          max_iter=1000, penalty='l1',
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2']},
             refit='AUC', return_train_score=True,
             scoring={'AUC': 'roc_auc', 'f1': 'f1'})

In [133]:
lgt_cv.cv_results_

{'mean_fit_time': array([0.06485963, 0.16904142, 0.09510107, 0.17513025, 0.1082942 ,
        0.17448442, 0.13067164, 0.1581207 , 0.1754122 , 0.15151753,
        0.28155913, 0.15985005, 0.30583873, 0.17462189, 0.59102445,
        0.15971925, 0.65745363, 0.17931962, 0.57870052, 0.16285987,
        0.9049072 , 0.16792064, 0.60123131, 0.21299865, 0.90693364,
        0.20629165, 0.8488121 , 0.19746249, 0.83693693, 0.19581327,
        0.58909695, 0.17949936, 0.64263039, 0.18019729, 0.76432927,
        0.18644621, 0.56470211, 0.20304995, 0.68067343, 0.17518659]),
 'std_fit_time': array([0.01249547, 0.01422762, 0.03926555, 0.01249407, 0.02959724,
        0.01984706, 0.05389558, 0.02748609, 0.07240349, 0.03307395,
        0.12357524, 0.02218903, 0.1038663 , 0.03345587, 0.32480777,
        0.03383108, 0.24845877, 0.02570257, 0.26711169, 0.02007337,
        0.46551409, 0.01660953, 0.25442435, 0.03418746, 0.76155952,
        0.03302161, 0.27947151, 0.040446  , 0.61335886, 0.02010662,
        0.315

In [134]:
print("The best parameters after grid search: ", lgt_cv.best_params_)

print("The best score is ", lgt_cv.best_score_.round(3))

The best parameters after grid search:  {'C': 4.281332398719396, 'penalty': 'l1'}
The best score is  0.946


In [135]:
y_pred = lgt_cv.predict(X_test)

In [136]:
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
#f1_score = f1_score(y_test, y_pred)

In [137]:
print("Precision is ", round(precision,3))
print("Recall is ", round(recall,3))
#print("F1 score is ", round(f1_score,3))

Precision is  0.483
Recall is  0.851


In [138]:
print("Classification report:\n", classification_report(y_test, y_pred))

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.91      0.94       981
           1       0.48      0.85      0.62       101

    accuracy                           0.90      1082
   macro avg       0.73      0.88      0.78      1082
weighted avg       0.94      0.90      0.91      1082



In [139]:
confusion_matrix(lgt_cv.predict(X_test),y_test)

array([[889,  15],
       [ 92,  86]])

In [140]:
cm = metrics.confusion_matrix(y_pred, y_test)
cm

array([[889,  15],
       [ 92,  86]])

## Evaluation Metrics Analysis

In [63]:
precision = cm[0,0]/(cm[0,0]+ cm[1,0])
#sensitivity <=> recall
sensitivity = cm[0,0]/(cm[0,0]+ cm[0,1])

specificity = cm[1,1]/(cm[1,1]+ cm[1,0])
recall = cm[0,0]/(cm[0,0]+ cm[0,1])

Geometric mean if both classes are important and more than 80 percent
in the majority class

In [64]:
g_mean = np.sqrt(sensitivity * specificity)

In [65]:
print("G-mean is ", g_mean.round(3))

G-mean is  0.845


In [66]:
def f_score(beta):
    f_sc = ((1+ np.power(beta, 2))*precision * recall)/\
    (np.power(beta, 2) * precision + recall)
    return f_sc

F1-Measure is used if both False Positives and False Negatives are equally important

In [67]:
#f_one_measure = (2 *precision * recall)/(precision + recall)

print("F1-Measure is ", round(f_score(1), 3))

F1-Measure is  0.958


F0.5-Measure is used if False Positives are more important than False Negatives

In [68]:
print("F0.5-Measure is ", round(f_score(0.5), 3))

F0.5-Measure is  0.968


F2-Measure is used if False Negatives are more important than False
Positives

In [69]:
print("F2-Measure is ", round(f_score(2), 3))

F2-Measure is  0.948


In [71]:
print("Precision is ", precision.round(3))
print("Recall (Sensitivity) is ", recall.round(3))
print("Specificity is ", specificity.round(3))
#print("Accuracy is ", accuracy.round(3)*100)

Precision is  0.975
Recall (Sensitivity) is  0.941
Specificity is  0.758


In [141]:
C_best = 4.28

## Feature Importance

In [142]:
lgt_best = LogisticRegression(C = C_best, penalty = "l1",max_iter = 1000,
                         solver = 'liblinear', \
                         class_weight = weights)

In [143]:
lgt_best.fit(X_train, y_train)

LogisticRegression(C=4.28, class_weight={0: 0.55, 1: 5.35}, max_iter=1000,
                   penalty='l1', solver='liblinear')

In [144]:
pr = lgt_best.predict(X_test)

In [145]:
#more important than just predict

#lgt_best.predict_proba(X_test)

In [146]:
print(lgt_best.score(X_train, y_train))

0.8911737523105361


In [147]:
confusion_matrix(lgt_best.predict(X_test),y_test)

array([[888,  15],
       [ 93,  86]])

In [148]:
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [149]:
sel_ = SelectFromModel(lgt_best)
sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=LogisticRegression(C=4.28,
                                             class_weight={0: 0.55, 1: 5.35},
                                             max_iter=1000, penalty='l1',
                                             solver='liblinear'))

In [150]:
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True])

In [151]:
selected_features = X_train.columns[(sel_.get_support())]
selected_features

Index(['DurationMedianIP', 'DurationMeanIP', 'DurationMedianOP',
       'DurationMeanOP', 'ChronCondsMedian', 'ChronCondsMean', 'ChronConsMode',
       'PtAgeMedian', 'PtAgeMean', 'NumPtsIP', 'NumPtsOP', 'NumStatesIP',
       'NumStatesOP', 'SharePhysician', 'MergeAvgClaimReimbursed',
       'NumTopCodeClaim', 'DupClaimNum_IP', 'DupClaimRatio_IP',
       'DupClaimStatesNum_IP', 'DupClaimProviderNum_IP',
       'DupClaimStartDtNum_IP', 'DupClaimNum_OP', 'DupClaimRatio_OP',
       'DupClaimStatesNum_OP', 'DupClaimProviderNum_OP',
       'DupClaimStartDtNum_OP', 'AvgDuration', 'NumOfPt', 'ClmPerPt',
       'TotalRev', 'RevPerPt', 'ClmNoPhy', 'ClmAllPhy', 'ClmNoProc',
       'ClmPerDt', 'Serves_Both', 'Serves_OP', 'DupClaimMultiState_IP_no',
       'DupClaimMultiState_IP_yes', 'DupClaimMultiProvider_IP_no',
       'DupClaimMultiProvider_IP_yes', 'DupClaimMultiStartDt_IP_no',
       'DupClaimMultiStartDt_IP_yes', 'DupClaim_OP_no', 'DupClaim_OP_yes',
       'DupClaimMultiProvider_OP_no', 'Du

In [152]:
print('Total number of features: {}'.format((X_train.shape[1])))
print('Features selected by Lasso: {}'.format(len(selected_features)))
print('Features not selected by Lasso: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

Total number of features: 55
Features selected by Lasso: 49
Features not selected by Lasso: 6


In [153]:
#lgt_best = LogisticRegression(C = 3792.69, penalty = "l2",max_iter = 1000,
#                         solver = 'liblinear', \
#                         class_weight = weights)

In [154]:
#the score here is accuracy (number of correct guesses divided by the total guesses)
#print(lgt_best.score(X_train, y_train))

In [155]:
print(lgt_best.intercept_)

[-0.41542942]


In [156]:
print(lgt_best.coef_)

[[-5.85457122e-01  5.69615986e-01 -9.45548557e-02  6.38546875e-03
  -3.56637703e-02 -3.70900520e-01 -9.49712538e-03  1.23135050e-02
   3.84966709e-03 -1.31062276e-01 -8.47068962e-03 -8.43523937e-04
   1.03177263e-01  4.76881503e-01 -1.01607942e-04  8.08614768e-02
   1.42257595e+00 -2.71676990e+00  4.56027994e-01  2.87472567e-02
  -1.17843957e+00  1.25808014e-02 -3.41542088e-01  2.59768677e-02
   2.44824235e-04 -2.71834473e-03  9.05550251e-02 -4.45101685e-03
   1.39209307e-03  4.25462952e-01  1.37174473e-05  1.38831524e-04
  -9.84481875e-02 -3.69948925e-02  8.24955960e-04  7.61946295e-02
   6.58395152e-02 -6.74824860e-01 -1.12296784e+00 -1.12667884e+00
  -6.24606517e-03 -2.93102451e-03 -2.44083923e-01 -1.01405638e-01
  -4.33967545e-02 -6.65920017e-03 -1.97177541e-01 -1.68039368e-03
  -2.61410483e-01 -3.30979417e-01 -1.79558149e-01 -7.29685108e-02
  -3.36777278e-01  0.00000000e+00 -2.11549828e-01]]


## Stochastic Gradient Descent

In [157]:
from sklearn.linear_model import SGDClassifier

In [158]:
sg = SGDClassifier(loss = 'log', max_iter = 1e4, l1_ratio = 1, \
                  alpha= 1e-4, shuffle = False)

In [159]:
sg.fit(X_train,y_train)

SGDClassifier(l1_ratio=1, loss='log', max_iter=10000.0, shuffle=False)

In [160]:
sg.score(X_train,y_train)

0.8807763401109058

In [161]:
sg.fit(X_test,y_test)

SGDClassifier(l1_ratio=1, loss='log', max_iter=10000.0, shuffle=False)

In [162]:
sg.score(X_test, y_test)

0.9066543438077634

In [163]:
np.random.seed(0)
sg.set_params(shuffle = True)

SGDClassifier(l1_ratio=1, loss='log', max_iter=10000.0)